In [5]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright (C) 2009-2018 the sqlparse authors and contributors
# <see AUTHORS file>
#
# This example is part of python-sqlparse and is released under
# the BSD License: https://opensource.org/licenses/BSD-3-Clause
#
# This example illustrates how to extract table names from nested
# SELECT statements.
#
# See:
# https://groups.google.com/forum/#!forum/sqlparse/browse_thread/thread/b0bd9a022e9d4895

import sqlparse
from sqlparse.sql import IdentifierList, Identifier
from sqlparse.tokens import Keyword, DML


def is_subselect(parsed):
    if not parsed.is_group:
        return False
    for item in parsed.tokens:
        if item.ttype is DML and item.value.upper() == 'SELECT':
            return True
    return False


def extract_from_part(parsed):
    from_seen = False
    for item in parsed.tokens:
        if from_seen:
            if is_subselect(item):
                for x in extract_from_part(item):
                    yield x
            elif item.ttype is Keyword:
                return
            else:
                yield item
        elif item.ttype is Keyword and item.value.upper() == 'FROM':
            from_seen = True


def extract_table_identifiers(token_stream):
    for item in token_stream:
        if isinstance(item, IdentifierList):
            for identifier in item.get_identifiers():
                yield identifier.get_name()
        elif isinstance(item, Identifier):
            yield item.get_name()
        # It's a bug to check for Keyword here, but in the example
        # above some tables names are identified as keywords...
        elif item.ttype is Keyword:
            yield item.value


def extract_tables(sql):
    stream = extract_from_part(sqlparse.parse(sql)[0])
    return list(extract_table_identifiers(stream))


if __name__ == '__main__':
    sql = """
    select    distinct e.tags["asup_id"] as asup_id
        , e.asup_key
        , d.dvc_serial_no
        , e.log_name
        , e.event_date
        , e.sequence_id
        , e.severity
        , e.event_type
        , e.message
        , e.reason
        , e.diskName
        , e.tags
        , e.sys_hash
        , d.dvc_model
        , d.rg_id
        , d.dvc_type
        , d.dvc_label
        , dvc_primary_path
        , dvc_secondary_path
        , d.dt
from asuprep.ems_events_may2018 e
join asuprep.dvc_device_may2018 d on d.asup_id = e.tags["asup_id"]
where d.dvc_serial_no = 'S20LNWAGB09910'
    """

    tables = ', '.join(extract_tables(sql))
    print('Tables: {0}'.format(tables))


Tables: e


In [26]:
sql = """
    select    distinct e.tags["asup_id"] as asup_id
        , e.asup_key
        , d.dvc_serial_no
        , e.log_name
        , e.event_date
        , e.sequence_id
        , e.severity
        , e.event_type
        , e.message
        , e.reason
        , e.diskName
        , e.tags
        , e.sys_hash
        , d.dvc_model
        , d.rg_id
        , d.dvc_type
        , d.dvc_label
        , dvc_primary_path
        , dvc_secondary_path
        , d.dt
from asuprep.ems_events_may2018 e
join asuprep.dvc_device_may2018 d on d.asup_id = e.tags["asup_id"]
where d.dvc_serial_no = 'S20LNWAGB09910'"""

sql_vector = list(filter(lambda a: a != '', sql.upper().replace('\n', ' ').split(' ')))
print(sql_vector)

['SELECT', 'DISTINCT', 'E.TAGS["ASUP_ID"]', 'AS', 'ASUP_ID', ',', 'E.ASUP_KEY', ',', 'D.DVC_SERIAL_NO', ',', 'E.LOG_NAME', ',', 'E.EVENT_DATE', ',', 'E.SEQUENCE_ID', ',', 'E.SEVERITY', ',', 'E.EVENT_TYPE', ',', 'E.MESSAGE', ',', 'E.REASON', ',', 'E.DISKNAME', ',', 'E.TAGS', ',', 'E.SYS_HASH', ',', 'D.DVC_MODEL', ',', 'D.RG_ID', ',', 'D.DVC_TYPE', ',', 'D.DVC_LABEL', ',', 'DVC_PRIMARY_PATH', ',', 'DVC_SECONDARY_PATH', ',', 'D.DT', 'FROM', 'ASUPREP.EMS_EVENTS_MAY2018', 'E', 'JOIN', 'ASUPREP.DVC_DEVICE_MAY2018', 'D', 'ON', 'D.ASUP_ID', '=', 'E.TAGS["ASUP_ID"]', 'WHERE', 'D.DVC_SERIAL_NO', '=', "'S20LNWAGB09910'"]


In [48]:
import pandas as pd
import graph_tool.all as gt
g = gt.Graph(directed= True)
v_label = g.new_vertex_property("string")

alias_raw = []
if 'AS' in sql_vector:
    name = sql_vector[sql_vector.index('AS')-1]
    alias = sql_vector[sql_vector.index('AS') + 1]
    alias_raw.append({'name': name, 'alias': alias})

if 'JOIN' in sql_vector:
    reference_name = sql_vector[sql_vector.index('FROM')+1:sql_vector.index('JOIN')][0]
    reference_alias = sql_vector[sql_vector.index('FROM')+1:sql_vector.index('JOIN')][1]
    factor_name = sql_vector[sql_vector.index('JOIN')+1:sql_vector.index('ON')][0]
    factor_alias = sql_vector[sql_vector.index('JOIN')+1:sql_vector.index('ON')][1]
    alias_raw.append({'name': reference_name, 'alias': reference_alias})
    alias_raw.append({'name': factor_name, 'alias': factor_alias})
    
    v0 = g.add_vertex() # join
    v_label[v0] = 'from'
    v1 = g.add_vertex() # join
    g.add_edge(v0, v1)
    v_label[v1] = 'join'
    v2 = g.add_vertex() # ref table
    v_label[v2] = reference_name
    v3 = g.add_vertex() # fac table
    v_label[v3] = factor_name
    
    g.add_edge(v1, v2)
    g.add_edge(v1, v3)
    
    
    v4 = g.add_vertex() # COND
    v_label[v4] = 'COND'
    g.add_edge(v1, v4)
    
    op1 = sql_vector[sql_vector.index('ON')+1:sql_vector.index('WHERE')][0]
    cond = sql_vector[sql_vector.index('ON')+1:sql_vector.index('WHERE')][1]
    op2 = sql_vector[sql_vector.index('ON')+1:sql_vector.index('WHERE')][2]
    
    v5 = g.add_vertex() # op1
    v_label[v5] = op1
    g.add_edge(v4, v5)
    
    v6 = g.add_vertex() # operation
    v_label[v6] = cond
    g.add_edge(v4, v6)
    
    v7 = g.add_vertex() # op2
    v_label[v7] = op2
    g.add_edge(v4, v7)
    g.vertex_properties["label"] = v_label
    
gt.graph_draw(g, vertex_text=g.vp.label, vertex_font_size=8, 
              vertex_text_color = 'black', vertex_fill_color = 'white', vertex_color='black',
              output_size=(800, 800), output="query1.png")
    
alias_df = pd.DataFrame(data=alias_raw)
print(alias_df)

                         name    alias
0           E.TAGS["ASUP_ID"]  ASUP_ID
1  ASUPREP.EMS_EVENTS_MAY2018        E
2  ASUPREP.DVC_DEVICE_MAY2018        D
